In [1]:
import pandas as pd
df1=pd.read_csv("reviews.csv")
df=df1.head(5000)

In [2]:
df=df.drop("Time_submitted",axis=1)
df=df.drop('Total_thumbsup',axis=1)
df=df.drop('Reply',axis=1)
df

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
2,"This pop-up ""Get the best Spotify experience o...",4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1
...,...,...
4995,"Great app, makes life happier.",5
4996,Good quality and consistent play format..my go...,5
4997,"If I click to play music on PC, it shouldn't s...",3
4998,I love it until some@$$ hacked my account and ...,4


In [3]:
import re
import string

def urlRemove(review):
    review=re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'',review)
    review=review.strip().replace('\n', '')
    review="".join([i for i in review if i not in string.punctuation])
    review=re.sub(r'\s+', ' ', review)
    return review.lower()

In [4]:
df['Review1']=df['Review'].apply(lambda x: urlRemove(x))

In [5]:
#lemmatization

import nltk
from nltk.stem import WordNetLemmatizer

def lemmatizeReview(review):
    tokens=re.split(' ',review)
    stopwords=nltk.corpus.stopwords.words('english')
    tokens= [i for i in tokens if i not in stopwords]
    lemmatizer=WordNetLemmatizer()
    lemm_text = [lemmatizer.lemmatize(word) for word in tokens]
    lemm_text=" ".join(lemm_text)
    return lemm_text

In [6]:
df['New_Review']=df['Review1'].apply(lambda x: lemmatizeReview(x))
df=df.drop('Review',axis=1)
df=df.drop('Review1',axis=1)

In [7]:
x=df['New_Review']
y=df['Rating']

In [8]:
#splitting data into train and test sets
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=32)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf=TfidfVectorizer(ngram_range=(1,2))

In [10]:
x_train=tf_idf.fit_transform(x_train)
x_test=tf_idf.transform(x_test)

x_train=x_train.toarray()
x_test=x_test.toarray()

In [11]:
from sklearn.naive_bayes import GaussianNB

gnb=GaussianNB()
gnb.fit(x_train,y_train)

GaussianNB()

In [12]:
y_pred=gnb.predict(x_test)

In [13]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(y_pred,y_test)
con

array([[ 84,  22,  21,  15,  28],
       [ 19,   4,   6,   3,   2],
       [ 17,   4,  11,   6,  31],
       [ 12,   2,   7,  14,  52],
       [  9,   3,   9,  16, 103]], dtype=int64)

In [14]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_pred,y_test)
print('Accuracy Score = %0.2F'%(acc*100))

Accuracy Score = 43.20


In [15]:
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [16]:
y_pred=lr.predict(x_test)

In [17]:
con=confusion_matrix(y_pred,y_test)
con

array([[119,  27,  36,  15,  11],
       [  2,   1,   1,   0,   0],
       [  1,   2,   2,   0,   0],
       [  3,   0,   7,   8,   4],
       [ 16,   5,   8,  31, 201]], dtype=int64)

In [18]:
acc=accuracy_score(y_pred,y_test)
print('Accuracy Score = %0.2F'%(acc*100))

Accuracy Score = 66.20
